In [1]:
import pandas as pd
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
testing = np.load("../grids/points_256/random/train_data.npy")

In [5]:
testing[0:int(len(testing)*0.9),:,:].shape

(135492, 256, 3)

In [8]:
np.concatenate((testing[0:int(len(testing)*0.1),:,:], testing[0:int(len(testing)*0.2),:,:])).shape

(45163, 256, 3)

In [8]:
arr = np.array([[1, 1, 1, 0]])
np.where(arr > 0.5, 2, 0.5)

array([[2. , 2. , 2. , 0.5]])

In [2]:
#felt1_NS = pd.read_csv("../../data/AllColumns/felt1_NS_points_ac.csv")
#felt1_EW = pd.read_csv("../../data/AllColumns/felt1_EW_points_ac.csv")

# felt2_NS = pd.read_csv("../../data/AllColumns/felt2_NS_points_ac.csv")
felt2_EW = pd.read_csv("../../data/AllColumns/felt2_EW_points_ac.csv")

In [3]:
felt2_EW = felt2_EW.drop(["Date", "Time", "Filename", "THU", "TVU"], axis=1)

In [4]:
felt2_EW

,X,Y,Z,Longitude,Latitude,Quality,TravelTime,SVL,BearingAngle,TiltAngle,BeamNumber,ScanNumber,Accepted
0,0.00,6.22,36.67,5.818626,58.979585,14,0.09,1471.5,-0.71,0.01,81,495,1
1,0.01,6.41,36.73,5.818626,58.979587,14,0.06,1471.5,0.24,0.01,215,380,1
2,0.02,6.69,36.49,5.818625,58.979590,14,0.06,1471.5,0.25,0.01,216,380,1
3,0.05,6.28,36.64,5.818626,58.979586,14,0.07,1471.7,0.30,0.01,265,83,1
4,0.10,6.26,36.59,5.818627,58.979586,142,0.06,1471.7,-0.15,0.00,187,70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082808,144.78,114.37,28.34,5.821051,58.980617,142,0.05,1471.1,0.34,0.00,248,122,1
1082809,144.78,114.41,28.38,5.821051,58.980617,142,0.08,1471.2,0.86,0.01,334,99,1
1082810,144.79,114.64,28.40,5.821051,58.980619,142,0.05,1471.1,0.35,0.00,249,122,1
1082811,144.80,114.03,28.30,5.821052,58.980614,142,0.08,1471.2,0.86,0.01,333,99,1


In [10]:
felt2_EW.to_csv("../../data/AllColumns/felt2_EW_test_ac.csv", index=False)

In [5]:
felt2_EW = pd.read_csv("../../data/AllColumns/felt2_EW_test_ac.csv") # Felt2_EW

In [3]:
def area_of_dataframe(dataframe, x_start, x_last, y_start, y_last):
    area_dataframe = dataframe[
        (dataframe["X"] >= x_start) &
        (dataframe["X"] < x_last) & 
        (dataframe["Y"] >= y_start) & 
        (dataframe["Y"] < y_last)
    ]
    return area_dataframe

In [4]:
def visualize_area(data_area, voxel_size=0.02):
    _x = np.array(data_area.X)
    _y = np.array(data_area.Y)
    _z = np.array(data_area.Z)

    _red = np.array(np.zeros(len(data_area.X)))
    _green = np.array(np.zeros(len(data_area.X)))
    _blue = np.array(np.zeros(len(data_area.X)))

    acceptance = np.array(data_area.Accepted)
    for idx, val in enumerate(acceptance):
        
        if val > 0:
            _green[idx] = 65535
        else:
            _red[idx] = 65535

    pcd = o3d.geometry.PointCloud()

    pcd.points = o3d.utility.Vector3dVector(np.vstack((_x, _y, _z)).transpose())
    pcd.colors = o3d.utility.Vector3dVector(np.vstack((_red, _green, _blue)).transpose()/65535)

    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)

    o3d.visualization.draw_geometries([voxel_grid])

In [8]:
# Pick an area
xMax = np.max(felt2_EW.X)
yMax = np.max(felt2_EW.Y)
left_side = area_of_dataframe(felt2_EW, xMax*0.5, xMax*1, yMax*0, yMax*1)
right_side = area_of_dataframe(felt2_EW, xMax*0, xMax*0.5, yMax*0, yMax*1)

In [11]:
right_side.mean()

X                 42.661008
Y                 56.677222
Z                 13.131380
Longitude          5.819325
Latitude          58.980056
Quality           14.070333
TravelTime         0.041809
SVL             1471.583609
BearingAngle       0.012730
TiltAngle          0.004411
BeamNumber       196.694142
ScanNumber       260.361705
Accepted           0.993629
dtype: float64

In [12]:
left_side.mean()

X                102.576143
Y                 59.864370
Z                 14.049811
Longitude          5.820363
Latitude          58.980110
Quality           15.504065
TravelTime         0.042292
SVL             1471.479378
BearingAngle       0.037582
TiltAngle          0.004830
BeamNumber       201.845910
ScanNumber       279.613941
Accepted           0.980384
dtype: float64

In [9]:
visualize_area(right_side, voxel_size=0.04)